In [ ]:
#this script is used for geth data extraction
import json
import requests
import sched, time
import pandas as pd
from datetime import datetime

jslist=[]
s = sched.scheduler(time.time, time.sleep)
def do_something(sc): 
    response = requests.get("https://api.etherscan.io/api?module=proxy&action=eth_gasPrice&apikey=69SK4N44HS4YU9UB18RP9W9HQZ1SU3M1QC")
    todos = json.loads(response.text)
    now = datetime.now()
    current_time = now.strftime("%Y-%m-%d %H:%M:%S")
    jslist.append(str(todos)+str(current_time))
    print(str(todos)+str(current_time))
    s.enter(60, 1, do_something, (sc,))

s.enter(60, 1, do_something, (s,))
s.run()

In [ ]:
!pip install jsonschema==3.2.0
!pip install web3
!pip install hexbytes
!pip install eth_typing
!pip install eth_utils
#after compiling this, restart runtime is required

In [ ]:
#this script is used for ethereum gas prices data extraction 
import collections
import math
import operator
from typing import (
    Iterable,
    Sequence,
    Tuple,
)

from eth_typing import (
    ChecksumAddress,
)
from eth_utils import (
    to_tuple,
)
from eth_utils.toolz import (
    curry,
    groupby,
    sliding_window,
)
from hexbytes import (
    HexBytes,
)

from web3 import Web3
from web3._utils.math import (
    percentile,
)
from web3.exceptions import (
    InsufficientData,
    ValidationError,
)
from web3.types import (
    BlockNumber,
    GasPriceStrategy,
    TxParams,
    Wei,
)
from numpy import mean
from csv import writer
from datetime import datetime
w3 = Web3(Web3.HTTPProvider('https://mainnet.infura.io/v3/1e61f3c3d3474fa0b9cff25f1a567714'))
MinerData = collections.namedtuple(
    'MinerData',
    ['timestamp', 'num_blocks', 'min_gas_price', 'low_percentile_gas_price'])
Probability = collections.namedtuple('Probability', ['gas_price', 'prob'])
def _get_raw_miner_data(
    w3: Web3, sample_size: int
)-> Iterable[Tuple[ChecksumAddress, HexBytes,int, Wei]]:
    latest = w3.eth.getBlock(11024553, full_transactions=True)

    for transaction in latest['transactions']:
        # type ignored b/c actual transaction is TxData not HexBytes
        yield (latest['hash'],latest.timestamp, transaction['gasPrice'])  # type: ignore

    block = latest

    for _ in range(5):
        if block['number'] == 0:
            break

        # we intentionally trace backwards using parent hashes rather than
        # block numbers to make caching the data easier to implement.
        block = w3.eth.getBlock(block['parentHash'], full_transactions=True)
        for transaction in block['transactions']:
            # type ignored b/c actual transaction is TxData not HexBytes
            yield (block['hash'],block.timestamp, transaction['gasPrice'])  # type: ignore


def _aggregate_miner_data(
    raw_data:Iterable[Tuple[ChecksumAddress, HexBytes,int, Wei]]
)-> Iterable[MinerData]:
    data_by_miner = groupby(0, raw_data)
    print(data_by_miner)
    for bhash , miner_data in data_by_miner.items():
        _,timestamp, gas_prices = map(tuple, zip(*miner_data))
        with open('/content/drive/My Drive/blockdatatest.csv', 'a+') as write_obj:
        # Create a writer object from csv module
            csv_writer = writer(write_obj )
        # Add contents of list as last row in the csv file
            csv_writer.writerow([int(str(set(timestamp))),min(gas_prices)])
        yield MinerData(
            timestamp,
            min(gas_prices),
            max(gas_prices),
            mean(gas_prices)
            )
list(_aggregate_miner_data(_get_raw_miner_data(w3,5)))